## Tools

In [ ]:
%pip install tensorflow[and-cuda]

In [2]:
import numpy as np # math library
import pandas as pd # dataframe library
import matplotlib.pyplot as plt # plotting library
import seaborn as sns # graphing library that builds ontop of mathplot
import tensorflow as tf # machine learning library
from tensorflow import keras # computer vision library from tensorflow 
sns.set_style("darkgrid")

## Dataset
*Note: This is using a 5 class subset. The subset bird species are RUFOUS TREPE, HOUSE FINCH, D-ARNAUDS BARBET, OVENBIRD, ASIAN GREEN BEE EATER.
These 5 were choosen because they had the most available data out of the 500+ species in the orginial dataset.*

In [5]:
# training subset = RUFOUS TREPE, HOUSE FINCH, D-ARNAUDS BARBET, OVENBIRD, ASIAN GREEN BEE EATER
#csv format: class id,filepaths(relative),labels,data set(e.g. training,valid or test),scientific name
data = pd.read_csv("birds.csv")
data.head(10)

,class id,filepaths,labels,data set,scientific name
0,40.0,train/ASIAN GREEN BEE EATER/162.jpg,ASIAN GREEN BEE EATER,train,MEROPS ORIENTALIS
1,40.0,train/ASIAN GREEN BEE EATER/136.jpg,ASIAN GREEN BEE EATER,train,MEROPS ORIENTALIS
2,40.0,train/ASIAN GREEN BEE EATER/153.jpg,ASIAN GREEN BEE EATER,train,MEROPS ORIENTALIS
3,40.0,train/ASIAN GREEN BEE EATER/102.jpg,ASIAN GREEN BEE EATER,train,MEROPS ORIENTALIS
4,40.0,train/ASIAN GREEN BEE EATER/145.jpg,ASIAN GREEN BEE EATER,train,MEROPS ORIENTALIS
5,40.0,train/ASIAN GREEN BEE EATER/101.jpg,ASIAN GREEN BEE EATER,train,MEROPS ORIENTALIS
6,40.0,train/ASIAN GREEN BEE EATER/105.jpg,ASIAN GREEN BEE EATER,train,MEROPS ORIENTALIS
7,40.0,train/ASIAN GREEN BEE EATER/166.jpg,ASIAN GREEN BEE EATER,train,MEROPS ORIENTALIS
8,40.0,train/ASIAN GREEN BEE EATER/106.jpg,ASIAN GREEN BEE EATER,train,MEROPS ORIENTALIS
9,40.0,train/ASIAN GREEN BEE EATER/103.jpg,ASIAN GREEN BEE EATER,train,MEROPS ORIENTALIS


## Basic Exploratory Data Analysis (EDA)
Worry about the EDA stuff after you get Naive Bayes working, it seems to matter little to it.

In [ ]:
# Grouping by data set and plotting histograms for each feature/label
for dataset, subset in data.groupby('data set'):
    plt.figure(figsize=(12, 10))
    subset['labels'].hist(alpha=0.5, label=dataset)
    plt.title(f'Histogram for data set: {dataset}')
    plt.xlabel('Labels')
    plt.ylabel('Frequency')
    plt.legend()
    plt.show()

## Naive Bayes Implementation

In [27]:
#Step 1 Computer vision!
# loop through dataframe and load image into computer vision
for index, row in data.iterrows():
    # read data set column of the given row
    dataset = row['data set']
    # skip over non-training data(data hygiene protection) to avoid trainning on valid/test data
    if dataset != 'train':
        continue
    # read filepath col of the given row
    filepath = row['filepaths']
    # read label
    label = row['labels']
    # load image into computer vision using native image size(this data set has an universal image size)
    image = tf.keras.utils.load_img(filepath)
    # the image is 224x224x3 (width x height x color_channels)
    image_array = tf.keras.utils.img_to_array(image,data_format='channels_last')